In [2]:
import spacy
from collections import Counter

In [10]:
nlp = spacy.load("en_core_web_md")

def summarize(text, num_sentences=10):
    #Process the text throught the spacy npl
    doc = nlp(text)

    #get each token in the doc where it a word and isn't a common word like "the", "a", or "is"
    words = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]
    #Create a counter for each of the words to find the most common
    word_freq = Counter(words)

    #Sort by most common words
    max_freq = max(word_freq.values(), default=1)
    #normalize it to give it a score of most frequent words scale count to 0-1(Adds a weight)
    for word in word_freq:
        word_freq[word] = word_freq[word] / max_freq

    #Score sentence dictionary,( Basic Hashmap frequency algorithm)
    sentence_scores = {}
    # go through each sentence of each word in the doc again 
    for sent in doc.sents:
        for word in sents:
            if word.text.lower() in word_freq: # if the word is in the freq list add it's weight
                sentence_scores[sent] = sentence_scores.get(sent, 0) + word_freq[word.text.lower()]

    #get the top sentences with the highest score
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

    summary = " ".join([sent.text for sent in summary_suntences])

    return summary

In [11]:
from transformers import pipeline

ModuleNotFoundError: No module named 'transformers'